<a href="https://colab.research.google.com/github/victoribeir0/Projetos_Ciencia_de_Dados/blob/main/selecao_caracteristicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [6]:
# Caso acontece erro com o import do scikit-learn:
!pip uninstall scikit-learn -y
!pip install -U scikit-learn

Found existing installation: scikit-learn 1.0
Uninstalling scikit-learn-1.0:
  Successfully uninstalled scikit-learn-1.0
  Using cached scikit_learn-1.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (23.1 MB)


In [85]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

from sklearn.datasets import load_wine, load_breast_cancer, load_digits # Imports de diferentes datasets.
from sklearn.feature_selection import SelectKBest, VarianceThreshold, f_classif, SequentialFeatureSelector
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from sklearn import tree # Importa o modelo de classificação a ser usado (árvore de decisão)

## Definindo os dados

In [100]:
# Aqui pode ser carregado qualquer um dos datasets que foram importados.
dados = load_breast_cancer() 

# Define as entradas (variáveis de entrada).
X = dados.data 

# Define as saídas.
Y = dados.target 

# Define o nome de cada variável.
variaveis = dados.feature_names

# Normaliza os dados (opcional).
min_max = preprocessing.MinMaxScaler()
X = min_max.fit_transform(X)

# Mostra a descrição do dataset.
# print(dados.DESCR)

## Definindo o classificador
Essa função é usada para classificar os dados.

In [90]:
def arvore(X,Y):
  # Sepera os dados em dados de treino e dados de teste.
  x_treino, x_teste, y_treino, y_teste = train_test_split(X,
                                                          Y,
                                                          stratify=Y,
                                                          test_size=0.25,
                                                          random_state=1)
  
  # Cria uma instância do modelo de classificação.
  clf = tree.DecisionTreeClassifier()

  # Ajusta o modelo.
  clf.fit(x_treino,y_treino)

  # Testa e mede a acurácia do modelo com os dados de teste.
  y_pred = clf.predict(x_teste)
  return  100*np.round(accuracy_score(y_teste, y_pred, normalize=True),3)

## Seleção de características (ou variáveis)
Aqui serão apresentados três técnicas, todas do módulo de seleção de características do *scikit-learn*.
### 1. Seleção de variáveis baseada em variância



In [98]:
# Valor mínimo para as variâncias (esse valor é escolhido pelo usuário e depende do conjunto de dados, é importate testar com vários valores).
minimo = 0.02
vt = VarianceThreshold(minimo) 

# Redução das variáveis de acordo com a variância (X_novo terá menos variáveis que o X).
X_novo = vt.fit_transform(X) 

# Determina as variâncias para cada variável em X.
variancias = vt.fit(X,Y)

# Cria um DataFrame com as variâncias e para as variáveis.
df_scores = pd.DataFrame(variancias.variances_)
df_var = pd.DataFrame(variaveis)

# Concatena os dois DataFrames e adiciona os nomes das colunas.
df_var = pd.concat([df_var,df_scores], axis=1)
df_var.columns = ['Característica','Variância']

# Mostra o DataFrame com todas a características e suas variâncias.
# print(df_var)
print('\n')

# Mantém somente as características que são maiores que o minimo e remove as outras.
filtro = df_var['Variância'] >= minimo
df_var.where(filtro, inplace = True)
df_var = df_var.dropna()

# Mostra o DataFrame com somentes as características que possuem variâncias maiores que o limite.
print(df_var.nlargest(X_novo.shape[1],'Variância'))

# Mostra o número inicial de variáveis e o número final (reduzido).
print('\n')
print('Número inicial de variáveis: ' + str(X.shape[1]))
print('Número final de variáveis: ' + str(X_novo.shape[1]))
print('\n')

# Mostra os resultados obtidos, com os dados não reduzidos e com os dados reduzidos.
print('Acurácia:')
print('Acurácia (dados não reduzidos): ' + str(arvore(X,Y)) + '% | ' + 'Acurácia (dados reduzidos): ' + str(arvore(X_novo,Y)) + '%')
print('Classificador: Árvore de Decisão')



            Característica  Variância
27    worst concave points   0.050934
7      mean concave points   0.037128
6           mean concavity   0.034827
20            worst radius   0.029512
2           mean perimeter   0.028146
22         worst perimeter   0.027957
0              mean radius   0.027769
26         worst concavity   0.027718
21           worst texture   0.026787
5         mean compactness   0.026195
25       worst compactness   0.023261
24        worst smoothness   0.022694
3                mean area   0.022244
9   mean fractal dimension   0.022073
1             mean texture   0.021119


Número inicial de variáveis: 30
Número final de variáveis: 15


Acurácia:
Acurácia (dados não reduzidos): 93.7% | Acurácia (dados reduzidos): 93.7%
Classificador: Árvore de Decisão


### 2. Seleção baseada em scores de variáveis (SelectKBest)



In [99]:
# Inicia o seletor de variáveis, k = número de variáveis que serão selecionadas.
K = 6
k_melhores = SelectKBest(f_classif, k=K)

# Redução das variáveis de acordo com a variância (X_novo terá menos variáveis que o X).
X_novo = k_melhores.fit_transform(X,Y)

# Determina o score para as variáveis.
scores = k_melhores.fit(X,Y)

# Cria um DataFrame com os scores e para as variáveis.
df_scores2 = pd.DataFrame(scores.scores_)
df_var2 = pd.DataFrame(variaveis)

# Concatena os dois DataFrames e adiciona os nomes das colunas.
df_var2 = pd.concat([df_var2,df_scores2], axis=1)
df_var2.columns = ['Característica','Score']

# Mostra o DataFrame com todas a características e suas variâncias.
# print(df_var2)
print('\n')

# Mostra o DataFrame com somentes as k características com maior score.
print(df_var2.nlargest(K,'Score'))
print('\n')

# Mostra os resultados obtidos, com os dados não reduzidos e com os dados reduzidos.
print('Acurácia:')
print('Acurácia (dados não reduzidos): ' + str(arvore(X,Y)) + '% | ' + 'Acurácia (dados reduzidos): ' + str(arvore(X_novo,Y)) + '%')
print('Classificador: Árvore de Decisão')



          Característica       Score
27  worst concave points  964.385393
22       worst perimeter  897.944219
7    mean concave points  861.676020
20          worst radius  860.781707
2         mean perimeter  697.235272
23            worst area  661.600206


Acurácia:
Acurácia (dados não reduzidos): 93.0% | Acurácia (dados reduzidos): 95.1%
Classificador: Árvore de Decisão


### 3. Seleção sequencial de varíaveis

In [ ]:
# Define o classificador a ser usado.
modelo = tree.DecisionTreeClassifier()

# Inicia o seletor de variáveis, n_features_to_select = número de variáveis que serão selecionadas.
sfs = SequentialFeatureSelector(modelo, n_features_to_select = 5)
sfs.fit(X, Y)

# Redução das variáveis de acordo com a variância (X_novo terá menos variáveis que o X).
X_novo = sfs.transform(X)

# Mostra quais variáveis foram selecionadas.
var = np.array(variaveis)
print('Variáveis selecionadas:')
print(var[sfs.get_support()])
print('\n')

# Mostra os resultados obtidos, com os dados não reduzidos e com os dados reduzidos.
print('Acurácia:')
print('Acurácia (dados não reduzidos): ' + str(arvore(X,Y)) + '% | ' + 'Acurácia (dados reduzidos): ' + str(arvore(X_novo,Y)) + '%')
print('Classificador: Árvore de Decisão')

Variáveis selecionadas:
['mean texture' 'mean compactness' 'texture error' 'worst radius'
 'worst concave points']


Acurácia:
Acurácia (dados não reduzidos): 94.0% | Acurácia (dados reduzidos): 95.0%
Classificador: Árvore de Decisão
